#### Imports

In [3]:
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Concatenate, concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.utils import np_utils, generic_utils
from keras.optimizers import SGD, RMSprop, Adadelta, Adagrad, Adam
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv2D, MaxPooling2D,Conv1D, MaxPooling1D
from keras.models import model_from_config
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.constraints import maxnorm
from keras.models import load_model
from keras.layers.convolutional import AveragePooling2D
from keras.callbacks import EarlyStopping


from sklearn.metrics import roc_curve, auc
import numpy as np

import random
import gzip
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.externals import joblib 
from scipy import sparse
import pdb
from math import  sqrt
from sklearn.metrics import roc_curve, auc
import theano
import subprocess as sp
import scipy.stats as stats
import argparse
import pandas as pd
import csv
from random import shuffle


/media/DATA/anaconda3/envs/keras/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/media/DATA/anaconda3/envs/keras/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/media/DATA/anaconda3/envs/keras/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into whi

In [4]:
from keras.layers.core import Lambda
from keras.models import Sequential
from tensorflow.python.framework import ops
import keras.backend as K
import tensorflow as tf
import numpy as np
import keras
import sys

#### Loading data

In [3]:
def clean(seq):
    seq = seq.split('\n')
    seq2 = ''
    for j in seq:
        seq2 = seq2 + j
    seq2 = seq2[2:len(seq2)-1]
    seq2 = seq2.split(' ')
    #print(seq2)
    seq3=[]
    for j in seq2:
        #print(j)
        if j=='':
            a=0
        else:
            seq3.append(float(j))
    return seq3

def load_data(nb_train = 165000,nb_val = 6000,nb_test = 5000):
    # load your data using this function
    verif = []

    verif = pd.read_csv("genomes/negatifs_m-m-str.csv", sep = "\t",header=None)
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
            

    neg1 = verif

    verif=[]

    verif = pd.read_csv("genomes/negatifs_divers-str.csv", sep = "\t",header=None)
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
    neg2 = verif
    verif=[]
    verif = pd.read_csv("genomes/negatifs_mouse_divers-str.csv", sep = "\t",header=None)
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
    neg3 = verif
    verif=[]

    verif = pd.read_csv("genomes/positifs_m-m-str.csv", sep = "\t",header=None)  
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
            
    pos1 = verif
    l = len(pos1)
    verif=[]

    verif = pd.read_csv("genomes/positifs_divers-str.csv", sep = "\t",header=None)  
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
            
    pos2 = verif
    l=l+len(pos2)
    verif=[]

    verif = pd.read_csv("genomes/positifs_mouse_divers-str.csv", sep = "\t",header=None)  
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
            
    pos3 = verif
    l=l+len(pos3)
    verif=[]

    bdd = np.concatenate((pos1,pos2,pos3,neg1,neg2,neg3))

    pos1 = []
    neg1 = []
    pos2 = []
    neg2 = []

    labels = np.zeros((len(bdd),1))

    bdd = np.concatenate((bdd,labels),axis=1)
    for i in range(l):
        bdd[i,4]=1
    labels=[]
    
    # shuffle total

    indices = np.arange(len(bdd))
    shuffle(indices)
    bdd = bdd[indices]
    indices=[]

    # 
    l = len(bdd)
    matrice = np.zeros((l,36,36,1)) #4
    matrice1 = np.zeros((l,36,36,1))
    matrice2 = np.zeros((l,36,4))
    matrice3 = np.zeros((l,36,4))
    matrice4 = np.zeros((l,36,4))
    matrice5 = np.zeros((l,36,4))

    for i in range(l):
        seq1 = bdd[i,0]
        seq2 = bdd[i,1]
        prob1 = np.array(clean(bdd[i,2]))
        prob2 = np.array(clean(bdd[i,3]))
        for j in range(len(seq1)):
            for k in range(len(seq2)):
                if (seq1[j]=='a' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='a'):
                    matrice[i,j,k,0] = 1
                    #matrice[i,j,k,1] = prob1[j]+prob2[k]
                elif (seq1[j]=='g' and seq2[k]=='c') or (seq1[j]=='c' and seq2[k]=='g'):
                    matrice[i,j,k,0] = 1
                    #matrice[i,j,k,1] = prob1[j]+prob2[k]
                elif (seq1[j]=='g' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='g'):
                    matrice[i,j,k,0] = 1
                    #matrice[i,j,k,1] = prob1[j]+prob2[k]
                matrice1[i,j,k,0] = prob1[j]+prob2[k]
        for j in range(len(seq1)):
            if seq1[j]=='a':
                matrice2[i,j,0] = 1
            elif seq1[j]=='u':
                matrice2[i,j,1] = 1
            elif seq1[j]=='g':
                matrice2[i,j,2] = 1
            elif seq1[j]=='c':
                matrice2[i,j,3] = 1
        for j in range(len(seq2)):
            if seq2[j]=='a':
                matrice3[i,j,0] = 1
            elif seq2[j]=='u':
                matrice3[i,j,1] = 1
            elif seq2[j]=='g':
                matrice3[i,j,2] = 1
            elif seq2[j]=='c':
                matrice3[i,j,3] = 1

    for i in range(36):
        matrice4[:,36-i-1,:] = matrice2[:,i,:]
    for i in range(36):
        matrice5[:,36-i-1,:] = matrice3[:,i,:]
    
    training = []
    training.append(matrice[:nb_train])
    training.append(matrice1[:nb_train])
    training.append(matrice2[:nb_train])
    training.append(matrice4[:nb_train])
    training.append(matrice3[:nb_train])
    training.append(matrice5[:nb_train])    
    
    validation = []
    validation.append(matrice[nb_train:nb_train+nb_val])
    validation.append(matrice1[nb_train:nb_train+nb_val])
    validation.append(matrice2[nb_train:nb_train+nb_val])
    validation.append(matrice4[nb_train:nb_train+nb_val])
    validation.append(matrice3[nb_train:nb_train+nb_val])
    validation.append(matrice5[nb_train:nb_train+nb_val])

    testing = []
    testing.append(matrice[-nb_test:])
    testing.append(matrice1[-nb_test:])
    testing.append(matrice2[-nb_test:])
    testing.append(matrice4[-nb_test:])
    testing.append(matrice3[-nb_test:])
    testing.append(matrice5[-nb_test:])

    labels = bdd[:,4]
    bdd = []
    y = labels[:nb_train]
    y = keras.utils.np_utils.to_categorical(y,2)
    val_y = labels[nb_train:nb_train+nb_val]
    val_y = keras.utils.np_utils.to_categorical(val_y,2)
    true_y = labels[-nb_test:]
    argtest=[]
    np.sum(y[:,1])
    return training, y, validation, val_y, testing, true_y

In [6]:
Data = load_data(150000,5000,20000)

26017
24222
37991
26023
24229
37993


#### Récupération du modèle

In [7]:
# paramètres 

matrixsize11 = 6
nbfilter11 = 24
matrixsize12 = 7
nbfilter12 = 4
matrixsize21 = 6
nbfilter21 = 24
matrixsize22 = 7
nbfilter22 = 4
nbfilter1 = 64
kernelsize = 7
nbfilters2 = 64
kernel_size2 = 7
Dense1 = 128
Dense2 = 512
Dense3 = 512
Dense4 = 128
Dense5 = 0

from keras.layers import merge
from keras import Model
k = matrixsize11
# init_weights
I = np.eye(k)
M = np.diag(np.ones(k-1),1) + np.diag(np.ones(k-1),-1) + np.eye(k)
I2 = np.zeros((k,k))
M2 = np.zeros((k,k))
for j in range(k):
    I2[:,j] = I[:,k-j-1]
    M2[:,j] = M[:,k-j-1]        
W = np.zeros((k,k,1,nbfilter11))
W[:,:,0,0] = I
W[:,:,0,1] = I2
W[:,:,0,2] = M
W[:,:,0,3] = M2
for j in range(4,nbfilter11):
    W[:,:,0,j] = np.random.randn(k,k)*0.2


k2 = matrixsize12
I = np.eye(k2)
M = np.diag(np.ones(k2-1),1) + np.diag(np.ones(k2-1),-1) + np.eye(k2)
I2=np.zeros((k2,k2))
M2=np.zeros((k2,k2))
for j in range(k2):
    I2[:,j] = I[:,k2-j-1]
    M2[:,j] = M[:,k2-j-1]   

Z = np.zeros((k2,k2,nbfilter11,nbfilter12))

for u in range(nbfilter12):
    Z[:,:,u,0] = I
    Z[:,:,u,1] = I2
    Z[:,:,u,2] = M
    Z[:,:,u,3] = M2
    for p in range(4,nbfilter12):
        Z[:,:,u,p]=np.random.randn(k2,k2)*0.3            

c2d1_input = keras.Input(shape=(36,36,1))
cnn2d1 = Conv2D(filters = nbfilter11, kernel_size=(k,k), padding='valid', input_shape=(36,36,1),strides=(1,1),weights=[W,np.zeros(nbfilter11)])(c2d1_input)
cnn2d1 = AveragePooling2D(pool_size=(3,3))(cnn2d1)
cnn2d1 = Conv2D(filters = nbfilter12, kernel_size = (k2, k2),strides=(1,1),padding='valid',weights=[Z,np.zeros(nbfilter12)])(cnn2d1)
cnn2d1 = Dropout(0.2)(cnn2d1)
cnn2d1 = Flatten()(cnn2d1)

c2d2_input = keras.Input(shape=(36,36,1))
cnn2d2 = Conv2D(filters = nbfilter11, kernel_size=(k,k), padding='valid', input_shape=(36,36,1),strides=(1,1),weights=[W,np.zeros(nbfilter11)])(c2d2_input)
cnn2d2 = AveragePooling2D(pool_size=(3,3))(cnn2d2)
cnn2d2 = Conv2D(filters = nbfilter12, kernel_size = (k2, k2),strides=(1,1),padding='valid',weights=[Z,np.zeros(nbfilter12)])(cnn2d2)
cnn2d2 = Dropout(0.2)(cnn2d2)
cnn2d2 = Flatten()(cnn2d2)

c1d1_input = keras.Input(shape=(36,4))
cnn1d1 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d1_input)
cnn1d1 = MaxPooling1D(pool_size=8)(cnn1d1)
cnn1d1 = Dropout(0.2)(cnn1d1)
cnn1d1 = Flatten()(cnn1d1)
#cnn1d1 = get_cnn_network_seq(nbfilter1,kernel_size1)(c1d1_input)
c1d2_input = keras.Input(shape=(36,4))
cnn1d2 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d2_input)
cnn1d2 = MaxPooling1D(pool_size=8)(cnn1d2)
cnn1d2 = Dropout(0.2)(cnn1d2)
cnn1d2 = Flatten()(cnn1d2)
#cnn1d2 = get_cnn_network_seq(nbfilter1,kernel_size1)(c1d2_input)
c1d3_input = keras.Input(shape=(36,4))
cnn1d3 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d3_input)
cnn1d3 = MaxPooling1D(pool_size=8)(cnn1d3)
cnn1d3 = Dropout(0.2)(cnn1d3)
cnn1d3 = Flatten()(cnn1d3)

c1d4_input = keras.Input(shape=(36,4))
cnn1d4 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d4_input)
cnn1d4 = MaxPooling1D(pool_size=8)(cnn1d4)
cnn1d4 = Dropout(0.2)(cnn1d4)
cnn1d4 = Flatten()(cnn1d4)

model21 = keras.layers.concatenate([cnn1d1,cnn1d2])
model21 = Dense(int(Dense2/2),kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model21)
model21 = BatchNormalization()(model21)
model21 = Activation('relu')(model21)
model21 = Dropout(0.3)(model21)

model22 = keras.layers.concatenate([cnn1d3,cnn1d4])
model22 = Dense(int(Dense2/2),kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model22)
model22 = BatchNormalization()(model22)
model22 = Activation('relu')(model22)
model22 = Dropout(0.3)(model22)

model2 = keras.layers.concatenate([model21,model22])
model2 = Dense(Dense2,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model2)
model2 = BatchNormalization()(model2)
model2 = Activation('relu')(model2)
model2 = Dropout(0.3)(model2)

model1 = keras.layers.concatenate([cnn2d1,cnn2d2])
model1 = Dense(Dense1,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model1)
model1 = Dropout(0.1)(model1)
model1 = BatchNormalization()(model1)
model1 = Activation('relu')(model1)

model = keras.layers.concatenate([model1,model2])
model = Dense(Dense3,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
model = BatchNormalization()(model)
model = Activation('relu')(model)
model = Dropout(0.2)(model)
model = Dense(Dense4,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
model = BatchNormalization()(model)
model = Activation('relu')(model)
model = Dropout(0.2)(model)

if Dense5>0:
    model = Dense(Dense5,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = Dropout(0.2)(model)

model = Dense(2,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
model = BatchNormalization()(model)
interaction_output = Activation('softmax')(model)
interaction = Model(inputs=[c2d1_input,c2d2_input,c1d1_input,c1d2_input,c1d3_input,c1d4_input],outputs=[interaction_output])

### Explication d'interaction par des motifs de séquences d'ARN primaires

In [1]:
def target_category_loss_output_shape(input_shape):
    return input_shape

def target_category_loss(x, category_index, nb_classes):
    return tf.multiply(x, K.one_hot([category_index], nb_classes))

def target_category_loss_output_shape(input_shape):
    return input_shape

def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + 1e-5)

def seq2augc(seq):
    string = ''
    for i in range(36):
        if seq[i][0]==1:
            string += 'a'
        elif seq[i][1]==1:
            string += 'u'
        elif seq[i][2]==1:
            string += 'g'
        elif seq[i][3]==1:
            string += 'c'
        else:
            a=0
    return string

def argumentsmax(argw):
    l=[]
    for r in argw:
        l.append(r)
    return l

In [10]:
nb_classes = 2
category_index = 1
target_layer = lambda x: target_category_loss(x, category_index, nb_classes)
interaction_output = Lambda(target_layer, output_shape = target_category_loss_output_shape)(interaction_output)
interaction = Model(inputs=[c2d1_input,c2d2_input,c1d1_input,c1d2_input,c1d3_input,c1d4_input],outputs=[interaction_output])
interaction.load_weights("model_finaldeep.h5")

In [9]:
[l.name for l in interaction.layers[4:8]]

['conv1d_1', 'conv1d_2', 'conv1d_3', 'conv1d_4']

Récupération des interactions véritables uniquement

In [11]:
indices_positifs = np.argwhere(Data[1][:,1]==1)[:,0]

Identification de motifs (très gourmand en temps de calcul, dû aux gradients)

In [18]:
motifs = []
for t in range(250):
        
    indice_1 = indices_positifs[t]
    sequence = Data[0][0][indice_1]
    matrice_sequence0 = np.zeros((1,36,36,1))
    matrice_sequence0[0,:,:,:] = sequence
    sequence = Data[0][1][indice_1]
    matrice_sequence1 = np.zeros((1,36,36,1))
    matrice_sequence1[0,:,:,:] = sequence
    sequence = Data[0][2][indice_1]
    matrice_sequence = np.zeros((1,36,4))
    matrice_sequence[0,:,:] = sequence
    sequence = Data[0][3][indice_1]
    matrice_sequence2 = np.zeros((1,36,4))
    matrice_sequence2[0,:,:] = sequence
    sequence = Data[0][4][indice_1]
    matrice_sequence3 = np.zeros((1,36,4))
    matrice_sequence3[0,:,:] = sequence
    sequence = Data[0][5][indice_1]
    matrice_sequence4 = np.zeros((1,36,4))
    matrice_sequence4[0,:,:] = sequence
    
    motif1 = []
    
    z1=0
    layer_name = 'conv1d_1'
    loss = K.sum(interaction.layers[-1].output)
    conv_output = [l for l in interaction.layers if l.name == layer_name][0].output
    grads = normalize(K.gradients(loss, conv_output)[0])
    gradient_function = K.function(interaction.input, [conv_output, grads])

    output, grads_val = gradient_function([matrice_sequence0,matrice_sequence1,matrice_sequence,matrice_sequence2,matrice_sequence3,matrice_sequence4])
    output, grads_val = output[0, :], grads_val[0, :, :]

    weights = np.mean(grads_val, axis = 0)
    cam = np.ones(output.shape[0 : 1], dtype = np.float32)

    for i, w in enumerate(weights):
        cam += w*output[:, i]

    cam = np.maximum(cam, 0)
    heatmap = cam / np.max(cam)
    args = argumentsmax(np.argwhere(heatmap==1)[:,0])
    if (len(args)==0):
        z1+=1
    else:    
        seq = seq2augc(matrice_sequence[0])
        args = (np.argwhere(heatmap>0.9)[:,0])
        for pos in argumentsmax(args):
            motif1.append(seq[pos:pos+7])
    layer_name = 'conv1d_2'
    loss = K.sum(interaction.layers[-1].output)
    conv_output = [l for l in interaction.layers if l.name == layer_name][0].output
    grads = normalize(K.gradients(loss, conv_output)[0])
    gradient_function = K.function(interaction.input, [conv_output, grads])

    output, grads_val = gradient_function([matrice_sequence0,matrice_sequence1,matrice_sequence,matrice_sequence2,matrice_sequence3,matrice_sequence4])
    output, grads_val = output[0, :], grads_val[0, :, :]

    weights = np.mean(grads_val, axis = 0)
    cam = np.ones(output.shape[0 : 1], dtype = np.float32)

    for i, w in enumerate(weights):
        cam += w * output[:, i]

    cam = np.maximum(cam, 0)
    heatmap = cam / np.max(cam)
    args = argumentsmax(np.argwhere(heatmap==1)[:,0])
    if (len(args)==0):
        z1+=1
    else:    
        seq = seq2augc(matrice_sequence2[0])
        args = (np.argwhere(heatmap>0.9)[:,0])
        for pos in argumentsmax(args):
            motif1.append(seq[pos:pos+7])
    
    #
    motif2 = []
    
    z2=0
    layer_name = 'conv1d_3'
    loss = K.sum(interaction.layers[-1].output)
    conv_output = [l for l in interaction.layers if l.name == layer_name][0].output
    grads = normalize(K.gradients(loss, conv_output)[0])
    gradient_function = K.function(interaction.input, [conv_output, grads])

    output, grads_val = gradient_function([matrice_sequence0,matrice_sequence1,matrice_sequence,matrice_sequence2,matrice_sequence3,matrice_sequence4])
    output, grads_val = output[0, :], grads_val[0, :, :]

    weights = np.mean(grads_val, axis = 0)
    cam = np.ones(output.shape[0 : 1], dtype = np.float32)

    for i, w in enumerate(weights):
        cam += w * output[:, i]

    cam = np.maximum(cam, 0)
    heatmap = cam / np.max(cam)
    
    args = (np.argwhere(heatmap==1)[:,0])
    if (len(args)==0):
        z2+=1
    else:    
        seq = seq2augc(matrice_sequence3[0])
        for pos in argumentsmax(args):
            motif2.append(seq[pos:pos+7])
    
    layer_name = 'conv1d_4'
    loss = K.sum(interaction.layers[-1].output)
    conv_output = [l for l in interaction.layers if l.name == layer_name][0].output
    grads = normalize(K.gradients(loss, conv_output)[0])
    gradient_function = K.function(interaction.input, [conv_output, grads])

    output, grads_val = gradient_function([matrice_sequence0,matrice_sequence1,matrice_sequence,matrice_sequence2,matrice_sequence3,matrice_sequence4])
    output, grads_val = output[0, :], grads_val[0, :, :]

    weights = np.mean(grads_val, axis = 0)
    cam = np.ones(output.shape[0 : 1], dtype = np.float32)

    for i, w in enumerate(weights):
        cam += w * output[:, i]

    cam = np.maximum(cam, 0)
    heatmap = cam / np.max(cam)
    
    args = (np.argwhere(heatmap==1)[:,0])
    if (len(args)==0):
        z2+=1
    else:    
        seq = seq2augc(matrice_sequence4[0])
        
        for pos in argumentsmax(args):
            motif2.append(seq[pos:pos+7])
    #
    if (z1>=2 or z2>=2):
        print('rien')
    else:
        motifs.append([motif1,motif2])
    #        
    print(t/250*100)

0.0
0.4
0.8
1.2
1.6
2.0
2.4
2.8000000000000003
3.2
3.5999999999999996
4.0
4.3999999999999995
4.8
5.2
5.6000000000000005
6.0
6.4
6.800000000000001
7.199999999999999
7.6
8.0
8.4
8.799999999999999
9.2
9.6
10.0
10.4
10.8
11.200000000000001
11.600000000000001
12.0
12.4
12.8
13.200000000000001
13.600000000000001
14.000000000000002
14.399999999999999
14.799999999999999
15.2
15.6
16.0
16.400000000000002
16.8
17.2
17.599999999999998
18.0
18.4
18.8
19.2
19.6
20.0
20.4
20.8
21.2
21.6
22.0
22.400000000000002
22.8
23.200000000000003
rien
23.599999999999998
rien
24.0
24.4
24.8
25.2
25.6
26.0
26.400000000000002
26.8
27.200000000000003
27.6
28.000000000000004
28.4
28.799999999999997
29.2
29.599999999999998
30.0
30.4
30.8
31.2
31.6
32.0
32.4
32.800000000000004
33.2
33.6
34.0
34.4
34.8
35.199999999999996
35.6
36.0
36.4
36.8
37.2
37.6
38.0
38.4
38.800000000000004
39.2
39.6
40.0
40.400000000000006
40.8
41.199999999999996
41.6
42.0
42.4
42.8
43.2
rien
43.6
44.0
44.4
44.800000000000004
45.2
45.6
46.0
rien
4

Exception ignored in: <bound method TF_Output.<lambda> of <tensorflow.python.pywrap_tensorflow_internal.TF_Output; proxy of <Swig Object of type 'TF_Output *' at 0x7f7dc7a6ef30> >>
Traceback (most recent call last):
  File "/home/louis_jeay/downloads/3-5-2-test/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 963, in <lambda>
    __del__ = lambda self: None
KeyboardInterrupt


KeyboardInterrupt: 

In [23]:
motifs

[[['auugcgg', 'uugcgga'], ['cccccgc', 'g']],
 [['aacuaga', 'uagaagu', 'aaguuau', 'auguca', 'ug', ''], ['cuccaag', 'auc']],
 [['uuggucu', 'uuagucu', 'agucuuu'], ['agc', 'cgaacag']],
 [['uugagcu', 'ugagcug', ''], ['agcua', 'guccga']],
 [['guccugu'], ['gaacggg', '']],
 [['uggaucg', 'ucggcuc', 'a'], ['uuauagg', 'auu']],
 [['uugacuc'],
  ['agaucgg',
   'aacuauu',
   'acuauuc',
   'cuauucg',
   'uauucga',
   'auucgag',
   'uucgaga',
   'ucgagaa',
   'cgagaag',
   'gagaagg']],
 [['uccacua', '', ''], ['a']],
 [['ucgcggg', 'ccgccgg', ''], ['agguccg', 'aggu']],
 [['auuucag', 'gacuccc', '', ''], ['guuguaa', '']],
 [['ucgccua'], ['gugcagu', '']],
 [['uaaagga'], ['ugcau', '']],
 [['ccgcccg', 'agguuag', 'gguuagc', 'guuagcc'], ['ggcucgg', 'cucggcu']],
 [['ugg',
   'gguguac',
   'guguacu',
   'uguacuu',
   'guacuuu',
   'uacuuug',
   'acuuugg',
   'cuuuggg',
   'uuuggga',
   'uugggac'],
  ['acuaguu']],
 [['ucggcgc'], ['ggguugu', '']],
 [['uucauca', 'uucauac', 'uaca', 'gacuacu'], ['aauccuu', 'guguaag']

In [20]:
motifs_save = np.array(motifs)

In [22]:
np.save('motifs.npy',motifs_save)

#### Récupération des motifs extraits pour 250 interactions*
*sur plus de 100 000 possibles (mais temps de calcul trop long pour faire les 100 000)

In [9]:
motifs = np.load('motifs.npy')

In [11]:
print(motifs)

[[list(['auugcgg', 'uugcgga']) list(['cccccgc', 'g'])]
 [list(['aacuaga', 'uagaagu', 'aaguuau', 'auguca', 'ug', ''])
  list(['cuccaag', 'auc'])]
 [list(['uuggucu', 'uuagucu', 'agucuuu']) list(['agc', 'cgaacag'])]
 [list(['uugagcu', 'ugagcug', '']) list(['agcua', 'guccga'])]
 [list(['guccugu']) list(['gaacggg', ''])]
 [list(['uggaucg', 'ucggcuc', 'a']) list(['uuauagg', 'auu'])]
 [list(['uugacuc'])
  list(['agaucgg', 'aacuauu', 'acuauuc', 'cuauucg', 'uauucga', 'auucgag', 'uucgaga', 'ucgagaa', 'cgagaag', 'gagaagg'])]
 [list(['uccacua', '', '']) list(['a'])]
 [list(['ucgcggg', 'ccgccgg', '']) list(['agguccg', 'aggu'])]
 [list(['auuucag', 'gacuccc', '', '']) list(['guuguaa', ''])]
 [list(['ucgccua']) list(['gugcagu', ''])]
 [list(['uaaagga']) list(['ugcau', ''])]
 [list(['ccgcccg', 'agguuag', 'gguuagc', 'guuagcc'])
  list(['ggcucgg', 'cucggcu'])]
 [list(['ugg', 'gguguac', 'guguacu', 'uguacuu', 'guacuuu', 'uacuuug', 'acuuugg', 'cuuuggg', 'uuuggga', 'uugggac'])
  list(['acuaguu'])]
 [list(['u

La fréquence de certains couples de motifs peut révéler des fonctions biologiques s'alliant, expliquant les interactions. Eventuellement à corréler aux familles des ARN à étudier. 